In [1]:
import pandas as pd
import numpy as np


# Import Y

In [6]:
likeall=pd.read_csv("./likeall.csv",header=None, names=['time','pageid','uid'],sep=',')

In [8]:
del(likeall)

In [9]:
commall=pd.read_csv("./commall.csv",header=None, names=['time','pageid','uid'],sep=',')

In [10]:
len(commall)

1164452

In [342]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)

In [343]:
#Transform to binary int
Y_label=np.asarray(Y['Yellow'], dtype="float64")
print Y_label.max()
threshold=1
np.place(Y_label, Y_label<threshold, 0)
np.place(Y_label, Y_label>=threshold, 1)
Y_label=Y_label.astype(int)

4.954545455


In [364]:
Y_label.sum()

119


# Import and Transform X

In [381]:
likeall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [382]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [9]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [383]:
new_metric=new_metric.fillna(value=0)

# Sparse to Dense

In [2]:
n=10400000
pre_metric=pd.DataFrame([])

In [3]:
num=3
m=1500000
n=1000000
while (m<1164452):
    pre_metric=pd.DataFrame([])
    while (n<m):
        print n
        likesub=likeall[n:n+10000]
        new_metric=likesub.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)
        del likesub
        merge=pd.concat( [new_metric,pre_metric])
        pre_metric=merge.groupby(level=0).sum()
        del new_metric
        n=n+10000
    pre_metric.to_csv(path_or_buf='./comm-metric%d.csv'%num)
    del pre_metric
    num=num+1
    m=n+500000

In [9]:
#save metric to csv
pre_metric.to_csv(path_or_buf='./u-p-metric21.csv')

# Feature Selection: Variance

In [374]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.09)
new_metric=selector.fit_transform(new_metric)

In [384]:
new_metric.shape

(600, 625)

In [376]:
X_selected=new_metric

# PCA

In [433]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
#new_metric=new_metric.fillna(value=0)
metric_pca=pca.fit_transform(new_metric)
#metric_pca=pca.fit_transform(pre_metric)

In [309]:
pca.explained_variance_ratio_.sum()

0.82758839287034192

In [372]:
print new_metric.shape
print metric_pca.shape

(600, 625)
(600L, 100L)


# NMF

In [391]:
from sklearn.decomposition import NMF
model = NMF(n_components=30, init='random',alpha=0.01,max_iter=500,tol=1e-4,random_state=0)
metric_nmf=model.fit_transform(new_metric)

In [365]:
#component-page
model.components_.shape
#user-component
metric_nmf.shape

(600L, 100L)


# Logistic Regression

In [194]:
from sklearn.linear_model import LogisticRegression
LogReg=LogisticRegression()

##  Feature Selection

### RFE

In [315]:
from sklearn.feature_selection import RFE
selector = RFE(LogReg, 46, step=1)
selector = selector.fit(metric_pca, Y_label)
selector.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True, False,  True,  True], dtype=bool)

In [316]:
X_selected=metric_pca[:,selector.support_]
X_selected.shape

(600L, 46L)

In [368]:
X_selected.shape

(600L, 37L)

### If not selection

In [434]:
X_selected=metric_pca

In [392]:
X_selected=metric_nmf

## Train Model

In [435]:
LogReg.fit(X_selected, Y_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## p value

In [318]:
from scipy import stats
sse = np.sum((LogReg.predict(X_selected) - Y_label) ** 2, axis=0) / float(X_selected.shape[0] - X_selected.shape[1])
se = np.array([
        np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X_selected.T, X_selected))))
    ])
t = LogReg.coef_ / se
p = 2 * (1 - stats.t.cdf(np.abs(t), Y_label.shape[0] - X_selected.shape[1]))
np.set_printoptions(suppress=True)
print p

[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.00237846  0.          0.
   0.00000001  0.          0.          0.          0.00196097  0.00025726
   0.          0.          0.00001629  0.          0.          0.
   0.03663612  0.          0.          0.02593059  0.          0.          0.
   0.00000101  0.          0.00237107  0.00000103  0.02805052  0.00032233
   0.          0.00000052  0.00000002  0.09764374  0.          0.00000002
   0.00517029  0.        ]]


## Evaluate Model

In [332]:
X=LogReg.predict(X_selected)
X.sum()

31

In [436]:
X=LogReg.predict(X_selected)
X=pd.concat([pd.DataFrame(X),pd.DataFrame(Y_label)],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
print Acc
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
print rate

1    0   1
0         
0  473  85
1    8  34
0.845


# Bayes

In [437]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(X_selected, Y_label)

GaussianNB(priors=None)

In [438]:
X=NB.predict(X_selected)
X=pd.concat([pd.DataFrame(X),pd.DataFrame(Y_label)],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
print Acc
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
print rate

1    0    1
0          
0  192    5
1  289  114
0.51


# Learning Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = cross_validation.ShuffleSplit(digits.data.shape[0], n_iter=100,
                                   test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

plt.show()